<a href="https://colab.research.google.com/github/chandrakanta-chaudhury/Misc/blob/master/apache_spark_dataprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

spark-3.0.0-preview2-bin-hadoop2.7/
spark-3.0.0-preview2-bin-hadoop2.7/data/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/sample_movielens_rati

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop3.2"

In [0]:
import findspark
findspark.init("/content/spark-3.0.0-preview2-bin-hadoop2.7")

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [18]:
#copy path /content/drive/My Drive/Irish_road_data1.zip
!unzip "/content/drive/My Drive/Irish_road_data1.zip"

Archive:  /content/drive/My Drive/Irish_road_data1.zip
  inflating: Irish_road_data1.csv    


In [0]:
file_loc="/content/Irish_road_data1.csv"

In [0]:
df = spark.read.csv(file_loc, inferSchema=True, header =True)

In [21]:
df.head(3)

[Row(cosit=997, year=2020, month=1, day=31, hour=1, minute=45, second=1, millisecond=0, minuteofday=105, lane=2, lanename='Test2', straddlelane=0, straddlelanename=None, class=5, classname='HGV_RIG', length=11.2, headway=3.55, gap=3.83, speed=69.0, weight=0.0, temperature=0, duration=0, validitycode=0, numberofaxles=0),
 Row(cosit=997, year=2020, month=1, day=31, hour=1, minute=45, second=3, millisecond=0, minuteofday=105, lane=1, lanename='Test1', straddlelane=0, straddlelanename=None, class=5, classname='HGV_RIG', length=11.3, headway=3.39, gap=3.73, speed=70.0, weight=0.0, temperature=0, duration=0, validitycode=0, numberofaxles=0),
 Row(cosit=997, year=2020, month=1, day=31, hour=1, minute=45, second=4, millisecond=0, minuteofday=105, lane=2, lanename='Test2', straddlelane=0, straddlelanename=None, class=2, classname='CAR', length=5.0, headway=3.13, gap=2.82, speed=69.0, weight=0.0, temperature=0, duration=0, validitycode=0, numberofaxles=0)]

In [22]:
df.printSchema()

root
 |-- cosit: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- second: integer (nullable = true)
 |-- millisecond: integer (nullable = true)
 |-- minuteofday: integer (nullable = true)
 |-- lane: integer (nullable = true)
 |-- lanename: string (nullable = true)
 |-- straddlelane: integer (nullable = true)
 |-- straddlelanename: string (nullable = true)
 |-- class: integer (nullable = true)
 |-- classname: string (nullable = true)
 |-- length: double (nullable = true)
 |-- headway: double (nullable = true)
 |-- gap: double (nullable = true)
 |-- speed: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- duration: integer (nullable = true)
 |-- validitycode: integer (nullable = true)
 |-- numberofaxles: integer (nullable = true)



In [0]:
df_class_usage=df.groupby(['classname']).count()

In [29]:
df_class_usage.head(3)

[Row(classname='CAR', count=824422),
 Row(classname='HGV_ART', count=61329),
 Row(classname='BUS', count=10110)]

In [0]:
import pyspark.sql.functions as F

In [0]:
tot = df.count()

In [37]:
df.groupBy("classname") \
  .count() \
  .withColumn('usage', (F.col('count') / tot) * 100 ) \
  .show()

+---------+------+--------------------+
|classname| count|               usage|
+---------+------+--------------------+
|      CAR|824422|   78.62308370884296|
|  HGV_ART| 61329|   5.848794792933266|
|      BUS| 10110|  0.9641656533867391|
|  HGV_RIG| 36983|  3.5269770879526976|
|     null|   107|0.010204324917149465|
|  CARAVAN|  3687|  0.3516200557900007|
|      LGV|108296|   10.32792122642634|
|    MBIKE|  3641| 0.34723314975085234|
+---------+------+--------------------+



In [0]:
#Calculate the highest and lowest hourly flows on M50 - show the hours and total number of vehicle counts.
#Filter Cosit values to get the M50 Category as follows: 
#lstm50 = [1012,1500,1501,1502,1508,1503,1509,1504,1505,1506,1507,15010,15011,15012]
#

In [0]:
filterlist = [1012,1500,1501,1502,1508,1503,1509,1504,1505,1506,1507,15010,15011,15012]
 

In [0]:
filter_M50 = df.filter(~df.cosit.isin(filterlist))

In [67]:
from pyspark.sql.functions import desc
filter_M50_desc=filter_M50.groupBy(["cosit","hour"]) \
  .count().sort(desc("count")) \
  .withColumnRenamed("count","vechicle_count") \
  .show()

+-----+----+--------------+
|cosit|hour|vechicle_count|
+-----+----+--------------+
| 1014|   8|          5332|
| 1014|  14|          5279|
| 1014|  16|          5235|
| 1014|  17|          5132|
| 1014|   7|          5006|
| 1073|  10|          4854|
| 1014|  12|          4737|
| 1014|   9|          4601|
| 1014|  18|          4564|
| 1014|  11|          4273|
| 1014|  10|          4235|
| 1014|  13|          4163|
| 1014|   6|          3950|
| 1014|  19|          3927|
| 1071|  10|          3896|
| 1071|  14|          3831|
| 1015|  16|          3788|
| 1034|  16|          3680|
| 1015|  18|          3653|
| 1071|  13|          3647|
+-----+----+--------------+
only showing top 20 rows



In [68]:
from pyspark.sql.functions import asc
filter_M50_Asc=filter_M50.groupBy(["cosit","hour"]) \
  .count().sort("count",ascending=True) \
  .withColumnRenamed("count","vechicle_count") \
  .show()

ERROR! Session/line number was not unique in database. History logging moved to new session 69
+-----+----+--------------+
|cosit|hour|vechicle_count|
+-----+----+--------------+
| 1161|   3|             1|
| 1701|   5|             1|
| 1031|   4|             1|
|20562|   2|             1|
|20301|  23|             1|
|20113|   4|             1|
| 1592|   2|             1|
| 3807|  23|             1|
| 3806|   3|             1|
| 1592|   3|             1|
|20671|  23|             1|
| 3806|   5|             1|
|20812|   4|             1|
| 1722|   3|             1|
|31021|   2|             1|
|20601|   5|             1|
|20861|  23|             1|
| 1594|   3|             1|
|20591|   2|             1|
| 3805|   4|             1|
+-----+----+--------------+
only showing top 20 rows

